In [1]:
from sentence_encoder import *
import itertools as it


In [2]:
import sys
import predict1

<type 'classobj'>
['', '../models/', '..', '/home/tirthankar/miniconda3/envs/jcdl/lib/python27.zip', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7/plat-linux2', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7/lib-tk', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7/lib-old', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7/lib-dynload', '/home/tirthankar/.local/lib/python2.7/site-packages', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7/site-packages', '/home/tirthankar/miniconda3/envs/jcdl/lib/python2.7/site-packages/IPython/extensions', '/home/tirthankar/.ipython']
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [3]:
#group reviews into sets of 3, each set for one paper
def collate(reviews):
    iters = [iter(reviews)]*3
    return np.array(map(lambda x:list(x), zip(*iters)))

In [4]:
data_padded_, label_scale_, aspects_ = predict1.prepare_data('../../data/iclr_2018')

# (papers, paper obj, reviews_all, review, no.reviews, reviews, decision), aspects_score = datapadded

# All papers should have only 3 reviews

In [5]:
(papers,paper_obj,reviews_all,_,revs_count,_,decision),_ = data_padded_ 


In [6]:
reviews_all = list(it.chain.from_iterable(reviews_all))

In [7]:
len(reviews_all)

2727

In [8]:
reviews_embedded = embed(reviews_all[:1000])

KeyboardInterrupt: 

In [10]:
reviews_embedded.shape

(2727, 159, 512)

In [11]:
len(papers)

909

In [12]:
papers_embedded = embed(papers[:400])

In [13]:
pe = embed(papers[400:600])

In [15]:
pe1 = embed(papers[600:])

In [23]:
print(papers_embedded.shape)
print(pe.shape)
print(pe1.shape)

In [16]:
pe1.shape

(309, 1494, 512)

In [18]:
pe = np.pad(pe, [(0,0),(0, 1494-1250), (0,0)], mode = 'constant', constant_values=0.0)

In [22]:
papers_embedded = np.pad(papers_embedded, [(0,0),(0, 1494-835), (0,0)], mode = 'constant', constant_values=0.0)

In [25]:
papers_emb = np.concatenate((papers_embedded, pe, pe1), axis = 0)

In [26]:
papers_emb.shape

(909, 1494, 512)

In [28]:
rev = collate(reviews_embedded)

In [29]:
rev.shape

(909, 3, 159, 512)

In [109]:
# revs = np.array(revs)

In [30]:
decision = np.array(decision).astype(int)

In [31]:
decision.shape

(909,)

### Model Definition

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [83]:
from __future__ import division

In [84]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        
        num_classes = 2
        
        self.p = nn.Sequential(
                            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 5),
                            nn.ReLU()
                            )
        self.r = nn.Sequential(
                            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 5),
                            nn.ReLU()
                            )
    
        #self.s1 = nn.Linear(4*525,rh1)

        self.l1 = nn.Linear(256, 100)
        
        self.l2 = nn.Linear(64,100)
        
        self.crossLinear = nn.Linear(100,100)
        
        self.softmax = nn.Softmax(dim=2)
        
        self.l3 = nn.Linear(400,200)
        self.l4 = nn.Linear(200,100)
        self.l5 = nn.Linear(100, num_classes)
        
#         self.multihead_attn = nn.MultiheadAttention(100, 1)
        
#         self.l3 = nn.Linear(2*100,100)
#         self.l4 = nn.Linear(100, num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = 0.7)
        
    def forward(self, paper, review, sentiment):  
        batch_size = paper.shape[0]
        r1, r2, r3 = review
        out_p = self.p(paper)
        out_p = F.max_pool1d(out_p, out_p.shape[2])

        out_r1 = self.r(r1)
        out_r1 = F.max_pool1d(out_r1, out_r1.shape[2])    #out_p/r shape = (batch_size, #filters, 1)
        
        out_r2 = self.r(r2)
        out_r2 = F.max_pool1d(out_r2, out_r2.shape[2])    #out_p/r shape = (batch_size, #filters, 1)
        
        out_r3 = self.r(r3)
        out_r3 = F.max_pool1d(out_r3, out_r3.shape[2])    #out_p/r shape = (batch_size, #filters, 1)
        
        
        
#         print out_p.view(batch_size, -1).shape
#         print out_r1.shape
        
#         out = torch.cat((out_p3, out_r3), dim = 1)         #out shape = (batch_size, num_filters*kernels, 1)
#        out = out_r3
        
        #r = self.s1(sentiment.view(batch_size, -1))
        #r = self.dropout(r)
        out_p = self.l1(out_p.view(batch_size,-1))
        out_r1 = self.l2(out_r1.view(batch_size,-1))
        out_r2 = self.l2(out_r2.view(batch_size,-1))
        out_r3 = self.l2(out_r3.view(batch_size,-1))
        
        query = out_p
        key1, key2, key3 = out_r1, out_r2, out_r3
        
        keys = torch.stack([key1, key2, key3], dim=1)  #keys shape = (batch_size, 3, dim)
#         print keys.shape
        BilinearAttn = torch.bmm(self.crossLinear(query).unsqueeze(1), keys.transpose(1,2))
#         print BilinearAttn.shape
        attnWts = self.softmax(BilinearAttn)   #attnWts shape = (batch_size,1,3)
    
    
        values = attnWts.transpose(1,2)*keys
#         print values.shape
        
        values = values.view(batch_size, -1)
        
        cross = torch.cat((out_p, values), dim = 1)
        
        
        out = self.relu(self.l3(cross))
        out = self.relu(self.l4(out))
        out = self.l5(out)
        
        
        #out = self.l1(out.view(batch_size, -1))
        #out = self.dropout(out)
        
    
        
        return out, attnWts

In [85]:
class dset(Dataset):
    def __init__(self, data, y_data):
        x1, x2 = data
        print x1.shape
        print x2.shape
        assert x1.shape[0] == x2.shape[0]
        self.len = x1.shape[0]
        self.x1_data = x1
        self.x2_data = x2
        self.y_data = y_data
    def __getitem__(self, index):
        return (self.x1_data[index,:,:], self.x2_data[index,:,:,:]), self.y_data[index] #, self.x3_data[index,:,:]), self.y_data[index]
    def __len__(self):
        return self.len

In [86]:
def load_data(Dset, batch_size, num_workers):
    loader = DataLoader(Dset, batch_size = batch_size, shuffle = False, num_workers = num_workers)
    return loader


In [87]:
trainD = dset((papers_emb, rev), decision)
trainloader = load_data(trainD, batch_size = 20, num_workers = 1)

In [88]:
model = net()

In [89]:
optimizer = torch.optim.Adam(model.parameters()) #,weight_decay = 0.0, momentum = 0.9, lr = 0.009)
loss = torch.nn.CrossEntropyLoss()

In [90]:
for epoch in range(20):
    epoch_loss = []
    epoch_acc = []
    for i, data in enumerate(trainloader,0):
        (paper, revs), decs = data   #paper = (batch_size, seq_len, dim) #revs = (batch_size, 3 , seq_len, dim)
        paper = paper.transpose(1,2).float()
        rev1 = revs[:,0,:,:].transpose(1,2).float()
        rev2 = revs[:,1,:,:].transpose(1,2).float()
        rev3 = revs[:,2,:,:].transpose(1,2).float()
        out, attnWts = model(paper, (rev1, rev2, rev3),_)
#        print attnWts
#         print decision
#         print out
#         print out
        pred = (torch.max(out, 1)[1].view(decs.size()).data == decs.data).sum()
#         print pred
        acc = (pred.item()/decs.size()[0])
        optimizer.zero_grad()
        los = loss(out,decs)
        los.backward()
        optimizer.step()
        
        epoch_loss.append(los.item())
        epoch_acc.append(acc)
#         break
    print("Epoch:{}, Loss:{}, Acc:{}".format(epoch,np.average(epoch_loss),np.average(epoch_acc)))
#     break

KeyboardInterrupt: 

In [91]:
test_p = papers_emb[:10,:,:]
test_r = rev[:10,:,:,:]
test_d = decision[:10]


In [92]:
testD = dset((test_p, test_r), test_d)
testloader = load_data(testD, batch_size = 10, num_workers = 1)

In [93]:
for data in testloader:
    (p, r), d = data
    pt = p.transpose(1,2).float()
    rt1 = r[:,0,:,:].transpose(1,2).float()
    rt2 = r[:,1,:,:].transpose(1,2).float()
    rt3 = r[:,2,:,:].transpose(1,2).float()
    out_t, attnWts_t = model(pt, (rt1, rt2, rt3),_)

In [94]:
attnWts_t

tensor([[[8.2664e-01, 5.5302e-02, 1.1806e-01]],

        [[9.9929e-01, 5.7948e-07, 7.0773e-04]],

        [[6.1960e-02, 9.1830e-01, 1.9742e-02]],

        [[5.9085e-03, 9.7904e-01, 1.5055e-02]],

        [[9.1499e-02, 8.9191e-01, 1.6596e-02]],

        [[3.5793e-02, 7.5099e-01, 2.1322e-01]],

        [[2.3788e-04, 4.8013e-01, 5.1963e-01]],

        [[2.8056e-02, 6.8387e-01, 2.8807e-01]],

        [[8.3396e-01, 1.4250e-03, 1.6462e-01]],

        [[7.7019e-01, 1.5070e-01, 7.9109e-02]]], grad_fn=<SoftmaxBackward>)

In [99]:
[review.__dict__ for review in paper_obj[6].__dict__['REVIEWS']]
    

[{'APPROPRIATENESS': None,
  'CLARITY': None,
  'COMMENTS': u'In this paper the authors present a model for more accurate Netflix recommendations (rating predictions, RMSE).  In particular, the authors demonstrate that a deep autoencoder, carefully tuned, can out-perform  more complex RNN-based models that have temporal information.  The authors examine how different non-linear activations, model size, dropout, and a novel technique called "dense re-feeding" can together improve DNN-based collaborative filtering.  Pros: - The accuracy results are impressive and a useful datapoint in how to build a DNN-based recommender.   - The dense re-feeding technique seems to be novel with incremental (but meaningful) benefits.    Cons: - Experimental results on only one dataset.  - Difficult to know if the results are generalizable.  ',
  'DATE': None,
  'IMPACT': None,
  'IS_ANNOTATED': None,
  'IS_META_REVIEW': None,
  'MEANINGFUL_COMPARISON': None,
  'ORIGINALITY': None,
  'OTHER_KEYS': None,
 